In [ ]:
!cp drive/MyDrive/hateful_memes.zip /content

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 49.9 MB/s 
     |████████████████████████████████| 6.6 MB 36.9 MB/s 
     |████████████████████████████████| 86 kB 4.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import pipeline, ViTFeatureExtractor, AutoFeatureExtractor
import numpy as np

nlp_features = pipeline("feature-extraction")
output = nlp_features('Deep learning is a branch of Machine learning')
print(torch.tensor(output).size()) # (Samples, Tokens, Vector Size)

# image_extractor = ViTFeatureExtractor()
# # y = image _extractor(dataset[0][1])['pixel_values']

# extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-50")

# print(extractor(dataset[0][1])["pixel_values"][0].flatten().shape)

# print(y[0].flatten().shape)

No model was supplied, defaulted to distilbert-base-cased (https://huggingface.co/distilbert-base-cased)
Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 10, 768])


In [ ]:
!mkdir memes

In [ ]:
%cd memes
!unzip hateful_memes.zip

Streaming output truncated to the last 5000 lines.
  inflating: hateful_memes/img/37640.png  
  inflating: hateful_memes/img/60718.png  
  inflating: hateful_memes/img/82945.png  
  inflating: hateful_memes/img/15387.png  
  inflating: hateful_memes/img/81294.png  
  inflating: hateful_memes/img/75498.png  
  inflating: hateful_memes/img/14072.png  
  inflating: hateful_memes/img/23806.png  
  inflating: hateful_memes/img/71826.png  
  inflating: hateful_memes/img/84307.png  
  inflating: hateful_memes/img/56241.png  
  inflating: hateful_memes/img/10572.png  
  inflating: hateful_memes/img/51248.png  
  inflating: hateful_memes/img/19230.png  
  inflating: hateful_memes/img/05619.png  
  inflating: hateful_memes/img/03268.png  
  inflating: hateful_memes/img/72491.png  
  inflating: hateful_memes/img/35902.png  
  inflating: hateful_memes/img/43658.png  
  inflating: hateful_memes/img/80613.png  
  inflating: hateful_memes/img/96071.png  
  inflating: hateful_memes/img/35916.png  
  i

In [ ]:
%cat  hateful_memes/dev_seen.jsonl

import json

with open("/content/memes/hateful_memes/train.jsonl") as f:
  json_list = list(f)

new_list = []
for json_string in json_list:
  json_dictionary = json.loads(json_string)
  new_list.append(json_dictionary)





{"id": "08291", "img": "img/08291.png", "label": 1, "text": "white people is this a shooting range"}
{"id": "46971", "img": "img/46971.png", "label": 1, "text": "bravery at its finest"}
{"id": "03745", "img": "img/03745.png", "label": 1, "text": "your order comes to $37.50 and your white privilege discount brings the total to $37.50"}
{"id": "83745", "img": "img/83745.png", "label": 1, "text": "it is time.. to send these parasites back to the desert"}
{"id": "80243", "img": "img/80243.png", "label": 1, "text": "mississippi wind chime"}
{"id": "05279", "img": "img/05279.png", "label": 1, "text": "knowing white people , that's probably the baby father"}
{"id": "01796", "img": "img/01796.png", "label": 1, "text": "life hack #23 how to get stoned with no weed"}
{"id": "53046", "img": "img/53046.png", "label": 1, "text": "you've heard of elf on a shelf, now get ready for"}
{"id": "82301", "img": "img/82301.png", "label": 1, "text": "cooooooooooooon!!!!"}
{"id": "31752", "img": "img/31752.pn

In [ ]:
print(len(new_list))

8500


In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.models.resnet import resnet152
from torch.utils.data import DataLoader, Dataset
import torch.functional as F
from torchvision.io import read_image
import torch
from PIL import Image


class MemeDataset(Dataset):
  def __init__(self, json_list):
    self.json_list = json_list
    self.yoloModel = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
  
  def __len__(self):
    return len(self.json_list)

  def __getitem__(self, idx):
    path_to_img = "/content/memes/hateful_memes/" + self.json_list[idx]["img"]
    img = Image.open(path_to_img)
    results = self.yoloModel(path_to_img)
    classes = results.pandas().xyxy[0].loc[:, "name"].values
    text = self.json_list[idx]["text"]
    label = self.json_list[idx]["label"]
    return classes, img, text, label


dataset = MemeDataset(new_list)

print(dataset[0])



Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-6-4 Python-3.7.13 torch-1.11.0+cu113 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


(array(['person', 'person', 'tie'], dtype=object), <PIL.PngImagePlugin.PngImageFile image mode=RGB size=265x400 at 0x7F1C8F2CA690>, 'its their character not their color that matters', 0)


In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.4 MB/s 
     |████████████████████████████████| 1.2 MB 36.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=74b832b6f9e8a39171e68a688e7cd5971c86fdb5d5454f4f42335194dbb715db
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["Oh no"]

model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(np.array(embeddings).shape)

Load pretrained SentenceTransformer: sentence-transformers/paraphrase-MiniLM-L6-v2
Use pytorch device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(1, 384)


In [ ]:
import torch.nn as nn


class LanguageAndVisionConcat(torch.nn.Module):
    def __init__(
        self,
        num_classes,
        loss_fn,
        language_feature_dim,
        vision_feature_dim,
        fusion_output_size,
        dropout_p,
        
    ):
        super(LanguageAndVisionConcat, self).__init__()
        self.language_module = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
        self.vision_module = resnet152(pretrained=True)
        self.vision_module.fc = nn.Linear(2048, language_feature_dim)
 
        self.fusion = torch.nn.Linear(
            in_features=(language_feature_dim + vision_feature_dim), 
            out_features=fusion_output_size
        )
        self.fc = torch.nn.Linear(
            in_features=fusion_output_size, 
            out_features=num_classes
        )
        self.loss_fn = loss_fn
        self.dropout = torch.nn.Dropout(dropout_p)
        
    def forward(self, text, image, label=None):
        text_features = torch.nn.functional.relu(
            torch.tensor(self.language_module.encode(text))
        )
        print(text_features.size())

        print(image.size)
        image_features = torch.nn.functional.relu(
            self.vision_module(image)
        )
        print(image_features.size())
        combined = torch.cat(
            [text_features, image_features], dim=1
        )
        fused = self.dropout(
            torch.nn.functional.relu(
            self.fusion(combined)
            )
        )
        logits = self.fc(fused)
        pred = torch.nn.functional.softmax(logits)
        loss = (
            self.loss_fn(pred, label) 
            if label is not None else label
        )
        return (pred, loss)

In [ ]:
model = LanguageAndVisionConcat(
            num_classes=2,
            loss_fn=torch.nn.CrossEntropyLoss(),
            language_feature_dim=384,
            vision_feature_dim=384,
            fusion_output_size=248,
            dropout_p=0.2,
        )

Load pretrained SentenceTransformer: sentence-transformers/paraphrase-MiniLM-L6-v2
Use pytorch device: cpu


In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


img/42953.png


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([1, 384])
<built-in method size of Tensor object at 0x7f1c90480b90>
torch.Size([1, 384])
(tensor([[0.49457, 0.50543]], grad_fn=<SoftmaxBackward0>), None)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
dataloader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=20)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
for (i, j) in enumerate(dataloader):
  print(j)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


img/91432.png
img/63187.png
img/76214.png
img/37214.png
img/43026.png
img/46739.png
img/02483.png
img/39504.png
img/57042.png
img/53420.png
img/53618.png
img/80942.png
img/09374.png
img/83206.png
img/01546.png
img/73104.png
img/47368.png
img/04356.png
img/31652.png
img/72981.png
img/37190.png
img/58974.png
img/35724.png
img/40629.png
img/54691.png
img/58679.png


KeyboardInterrupt: ignored